In [ ]:
__nbid__ = '0073'
__author__  = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>, Astro David Herrera <david.herrera@noirlab.edu>, Data Lab Team <datalab@noirlab.edu>'
__version__ = '20250424' # aaaammdd
__datasets__ = ['desi_dr1']
__keywords__ = ['query', 'DESI']

# Cómo consultar datos DESI DR1

## Tabla de contenido

* [Objetivos](#Goals)
* [Resumen](#Summary)
* [Descargo de responsabilidad y atribución](#Disclaimer-and-Attribution)
* [Configuración inicial](#Initial-Setup)
* [Contenido de la Base de Datos](#Contents-of-the-Database)
* [Autenticación](#Authentication)
* [Consultas simples](#Simple-Queries)
* [Unir mesas](#Joining-Tables)
* [Coincidencia de tablas de desplazamiento al rojo con datos de orientación](#Matching-Redshift-Tables-to-Targeting-Data)
* [Progreso de el survey](#Survey-Progress)
* [Usando q3c](#Using-q3c)
* [Recursos y referencias](#Resources-and-References)

## Objetivos

Demuestre una variedad de consultas utilizando el conjunto de datos `desi_dr1`.

## Resumen

Este notebook cubrirá los conceptos básicos del uso de la base de datos de producción espectroscópica DESI, `desi_dr1`, que se carga desde las salidas del canal DESI. Este notebook está destinado a cubrir la mayor cantidad de detalles posible del [tutorial introductorio del notebook] de DESI (https://github.com/desihub/tutorials/blob/main/database/spectroscopic-production-database.ipynb), aunque reescrito para el entorno de Data Lab. Para obtener más detalles, incluidas descripciones detalladas de las tablas, consulte la [página de acceso a la base de datos] de DESI (https://data.desi.lbl.gov/doc/access/database/).

## Descargo de responsabilidad y atribución

## # Descargos de responsabilidad

Tenga en cuenta que el uso de Astro Data Lab constituye su acuerdo con nuestros [Descargos de responsabilidad] mínimos (https://datalab.noirlab.edu/disclaimers.php).

## # Agradecimientos

Si utiliza **Astro Data Lab** en su investigación publicada, incluya el texto en la sección de Agradecimientos de su artículo:

_Esta investigación utiliza servicios o datos proporcionados por Astro Data Lab, que forma parte del programa Community Science and Data Center (CSDC) de NSF NOIRLab. NOIRLab es operado por la Asociación de Universidades para la Investigación en Astronomía (AURA), Inc. bajo un acuerdo de cooperación con la Fundación Nacional de Ciencias de EE. UU.

Si utiliza **SPARCL junto con la plataforma Astro Data Lab** (a través de JupyterLab, línea de comandos o interfaz web) en su investigación publicada, incluya este texto a continuación en la sección de Agradecimientos de su artículo:

_Esta investigación utiliza servicios o datos proporcionados por el Laboratorio de Análisis y Catálogo Recuperable de SPectra (SPARCL) y el Laboratorio de Datos Astro, que forman parte del Programa del Centro de Datos y Ciencia Comunitaria (CSDC) de NSF NOIRLab. NOIRLab es operado por la Asociación de Universidades para la Investigación en Astronomía (AURA), Inc. bajo un acuerdo de cooperación con la Fundación Nacional de Ciencias de EE. UU.

En cualquier caso **por favor cite los siguientes artículos**:

* Documento conceptual del Laboratorio de datos: Fitzpatrick et al., "El Laboratorio de datos de NOAO: una descripción general conceptual", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general de Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

Si se refiere a Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Análisis astrofísico habilitado por Jupyter utilizando plataformas informáticas cercanas a datos", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

Si publica en una revista de la AAS, agregue también la palabra clave: `\facility{Astro Data Lab}`

Y si está utilizando SPARCL, agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Actas de la conferencia de ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

La biblioteca NOIRLab mantiene [listas de agradecimientos adecuados](https://noirlab.edu/science/about/scientific-acknowledgments) para utilizar al publicar artículos que utilicen las instalaciones, los datos o los servicios del laboratorio.

Para este notebook específicamente, reconozca:

* Citas y agradecimientos de datos DESI: https://data.desi.lbl.gov/doc/acknowledgments/

## Configuración inicial

Esto simplemente importa todo lo que necesitamos y configura rutas y variables de entorno para que podamos encontrar cosas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties
from dl import queryClient as qc, authClient as ac

# software DESI
from desitarget import __version__ as desitarget_version
from desitarget.targetmask import desi_mask

print(f"desitarget=={desitarget_version}")

## Contenidos de la Base de Datos

## # Esquema

Todas las tablas están agrupadas en el esquema de base de datos `desi_dr1`.

## # Notas importantes

* Esta base de datos no contiene ningún espectro del cielo. Se excluyen los espectros del cielo tanto dirigidos deliberadamente (`targetid & 2**59 != 0`) como negativos (`targetid < 0`).
* La tabla `ztile` *solo* contiene cantidades derivadas de espectros *acumulativos* basados ​​en mosaicos en este momento.
* Cada tabla siguiente tiene una clave principal y un "identificador único". En algunos casos, la clave principal y el identificador único son el mismo.
En otros casos, el identificador único es un conjunto de varias columnas y la clave principal es un valor arbitrario compuesto por esas columnas.
Por lo general, pero no siempre, el identificador único también tendrá un índice "ÚNICO" separado de la clave principal.

## # Las mesas

* `fotometría`. Este contiene los datos fotométricos puros. Por lo general, esto se deriva de los datos del Tractor LS DR9, pero no todos los objetos *objetivos* tienen fotometría del Tractor.
- Cargado desde archivos `tractorphot` en [lsdr9-photometry VAC](https://data.desi.lbl.gov/doc/releases/dr1/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/tractorphot/tractorphot-potential-*-iron.fits`, https://data.desi.lbl.gov/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/tractorphot/
- Clave principal: `targetid`.
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#photometry).
* `objetivo`. Contiene los bits de orientación y otros datos generados por "desitarget".
- Cargado desde el archivo `targetphot` en [lsdr9-photometry VAC](https://data.desi.lbl.gov/doc/releases/dr1/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/targetphot-potential-*-iron.fits`, https://data.desi.lbl.gov/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/
- Identificador único: (`targetid`, `survey`, `tileid`).
- Clave principal: `id`, un número entero único y arbitrario compuesto por (`targetid`, `survey`, `tileid`).
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#target).
* `azulejo`. Contiene información sobre observaciones agrupadas por mosaico.
- Cargado desde `tiles-iron.fits` de nivel superior, https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/tiles-iron.fits
- Clave principal: `tileid`.
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#tile).
* `exposición`. Contiene información sobre exposiciones individuales.
- Cargado desde `exposures-iron.fits` de nivel superior https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/exposures-iron.fits, `EXPOSURES` HDU.
- Clave principal: `expid`.
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#exposure).
* `marco`. Contiene información sobre exposiciones individuales, pero desglosada por cámara.  Por lo general, aunque no siempre, habrá 30 fotogramas por exposición.
- Cargado desde `exposures-iron.fits` de nivel superior, https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/exposures-iron.fits, `FRAMES` HDU.
- Identificador único: (`expid`, `camera`).
- Clave primaria: `frameid`, compuesta por `expid` y una asignación de `camera` a un número entero arbitrario.
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#frame).
* `asignación de fibra`. Contiene información sobre las posiciones de las fibras.
- Cargado desde archivos de fibra de asignación en el [producto de mosaicos](https://data.desi.lbl.gov/public/dr1/target/fiberassign/tiles/tags/1.1/).  Se cargan todos los archivos de asignación de fibra correspondientes a los mosaicos en la tabla "tile".
- Identificador único: (`tileid`, `targetid`, `ubicación`).
- Clave principal: `id`, un número entero único y arbitrario compuesto por (`tileid`, `targetid`, `location`).
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#fiberassign).
* `potencial`. Contiene una lista de `targetid` que *podrían* haber sido apuntados en un mosaico determinado.
- Cargado desde la HDU `POTENTIAL_ASSIGNMENTS` en los mismos archivos de asignación de fibra mencionados anteriormente.
- Identificador único: (`tileid`, `targetid`, `ubicación`).
- Clave principal: `id`, un número entero único y arbitrario compuesto por (`tileid`, `targetid`, `location`).
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#potential).
*`zpix`. Contiene los corrimientos al rojo de la canalización agrupados por HEALPixel.
- Cargado desde la HDU `ZCATALOG` en el archivo de resumen `zall-pix`: `/global/cfs/cdirs/desi/public/dr1/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits`, https://https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits.
- Identificador único: (`targetid`, `survey`, `program`).
- Clave principal: `id`, un número entero único y arbitrario compuesto por (`targetid`, `survey`, `program`).
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#zpix).
* `ztil`. Contiene los desplazamientos al rojo de la canalización agrupados por mosaico de diversas formas.
- Cargado desde la HDU `ZCATALOG` en el archivo de resumen `zall-tilecumulative`: `/global/cfs/cdirs/desi/public/dr1/spectro/redux/iron/zcatalog/v1/zall-tilecumulative-iron.fits`, https://https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/zcatalog/v1/zall-tilecumulative-iron.fits.
- Identificador único: (`targetid`, `spgrp`, `spgrpval`, `tileid`).
- Clave principal: `id`, un número entero único y arbitrario compuesto por (`targetid`, `spgrp`, `spgrpval`, `tileid`).
- [Descripciones de columnas](https://data.desi.lbl.gov/doc/access/database/#ztile).

## # Relaciones de clave externa

* `desi_dr1.target.targetid` -> `desi_dr1.photometry.targetid`.
* `desi_dr1.target.tileid` -> `desi_dr1.tile.tileid`.
* `desi_dr1.exposure.tileid` -> `desi_dr1.tile.tileid`.
* `desi_dr1.frame.expid` -> `desi_dr1.exposure.expid`.
* `desi_dr1.fiberassign.targetid` -> `desi_dr1.photometry.targetid`.
* `desi_dr1.fiberassign.tileid` -> `desi_dr1.tile.tileid`.
* `desi_dr1.potential.targetid` -> `desi_dr1.photometry.targetid`.
* `desi_dr1.potential.tileid` -> `desi_dr1.tile.tileid`.
* `desi_dr1.zpix.targetid` -> `desi_dr1.photometry.targetid`.
* `desi_dr1.ztile.targetphotid` -> `desi_dr1.target.id`.
* `desi_dr1.ztile.targetid` -> `desi_dr1.photometry.targetid`.
* `desi_dr1.ztile.tileid` -> `desi_dr1.tile.tileid`.

## Autenticación

Se puede acceder a gran parte de la funcionalidad de Data Lab sin iniciar sesión explícitamente (el servicio utiliza un inicio de sesión anónimo). Pero algunas capacidades, por ejemplo guardar los resultados de sus consultas en su espacio de almacenamiento virtual, requieren un inicio de sesión (es decir, necesitará una cuenta de usuario registrada).

Si necesita iniciar sesión en Data Lab, descomente la celda a continuación y ejecútela:

In [ ]:
# desde getpass importar getpass
# token = ac.login(input("Ingrese el nombre de usuario: (+ENTER) "), getpass("Ingrese la contraseña: (+ENTER) "))
# ac.quiénAmI()

## Consultas simples

En la mayoría de los ejemplos siguientes, incluimos el comando SQL sin formato equivalente que corresponde a la consulta.

## # Exposiciones, Noches, Azulejos

Aquí hay algunas consultas simples que demuestran conexiones simples entre noches, exposiciones y mosaicos.

## ## ¿Cuántas fichas hay?

```SQL
SELECCIONE CONTAR (tileid) DE desi_dr1.tile;
```

In [ ]:
response = qc.query(sql='SELECT COUNT(tileid) FROM desi_dr1.tile;', fmt='pandas', timeout=600)
response

## ## ¿En qué noches se observó un mosaico en particular?

```SQL
SELECCIONE noche, expid DESDE desi_dr1.exposure DONDE Tileid = 100;
```

In [ ]:
response = qc.query(sql='SELECT night, expid FROM desi_dr1.exposure WHERE tileid = 100;', fmt='pandas', timeout=600)
response

## ## ¿Qué mosaicos se observaron en una noche?

Una noche de observación DESI está representada por un número entero con el formato "AAAAMMDD".
```SQL
SELECCIONE Tileid, survey, programa DESDE desi_dr1.exposure DONDE noche = 20210115;
```

In [ ]:
response = qc.query(sql='SELECT tileid, survey, program FROM desi_dr1.exposure WHERE night = 20210115;', fmt='pandas', timeout=600)
response

## # Seleccionar objetivos ELG

A continuación se muestra una demostración sencilla de cómo encontrar objetivos específicos utilizando los valores de máscara de bits de destino. En este caso `desi_mask.ELG == 2**1`.

```SQL
SELECCIONE * de desi_dr1.target DONDE (desi_target & 2) != 0;
```

In [ ]:
q = f"SELECT * FROM desi_dr1.target WHERE (desi_target & {desi_mask.ELG:d}) != 0 LIMIT 10;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## ## Ejercicio

* ¿Cuántos objetos en la tabla `zpix` tienen `spectype` 'GALAXY'?

## # Corrimiento al rojo y clasificación

Filtrado de consultas simple sobre valores de cadena. Hay muchas estrellas, por lo que esta consulta tiene un "LÍMITE 20".

```SQL
SELECCIONE tipo de especificación, subtipo, z DESDE desi_dr1.zpix DONDE tipo de especificación = 'ESTRELLA' Y subtipo! = '' LÍMITE 20;
```

In [ ]:
response = qc.query(sql="SELECT spectype, subtype, z FROM desi_dr1.zpix WHERE spectype = 'STAR' AND subtype != '' LIMIT 20;", fmt='pandas', timeout=600)
response

## Unir mesas

## # Una unión sencilla

Veamos las noches y exposiciones en las que se observó un "TARGETID" en particular.

```SQL
SELECCIONE f.tileid, e.expid, e.night DESDE desi_dr1.fiberassign AS f ÚNASE a desi_dr1.exposure COMO e ON f.tileid = e.tileid DONDE f.targetid = 933811403620352;
```

In [ ]:
q = """SELECT f.tileid, e.expid, e.night
FROM desi_dr1.fiberassign AS f
JOIN desi_dr1.exposure AS e ON f.tileid = e.tileid
WHERE f.targetid = 933811403620352;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## # Otra unión sencilla

En este caso, veremos el flujo fotométrico y el corrimiento al rojo medido. `ORDENAREMOS`(`POR`) los resultados y `LIMITAREMOS` la consulta para mantener los números manejables.

```SQL
SELECCIONE p.*, z.* DE desi_dr1.photometry COMO p ÚNASE a desi_dr1.zpix COMO z ON p.targetid = z.targetid ORDENAR POR z.z, p.flux_g LÍMITE 50;
```

In [ ]:
q = """SELECT p.*, z.*
FROM desi_dr1.photometry AS p
JOIN desi_dr1.zpix AS z ON p.targetid = z.targetid
ORDER BY z.z, p.flux_g LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

In [ ]:
# Evite advertencias sobre valores no válidos en np.log10().
w = (response.flux_g.values > 0) & (response.flux_r.values > 0)
g_minus_r = np.log10(response.flux_r.values[w]/response.flux_g.values[w])
r_minus_z = np.log10(response.flux_z.values[w]/response.flux_r.values[w])
redshift = np.array(response.z.values[w])
fig, axes = plt.subplots(1, 1, figsize=(5, 5), dpi=100)
p = axes.plot(g_minus_r, r_minus_z, 'k.')
foo = axes.set_xlim([-0.2, 1.0])
foo = axes.set_ylim([-0.2, 1.0])
foo = axes.set_aspect('equal')
foo = axes.set_xlabel('$g-r$')
foo = axes.set_ylabel('$r-z$')

## ## Ejercicio

* Crear un gráfico color-color para objetos seleccionados como QSO y confirmados espectroscópicamente como tales.

## # Una unión más complicada

Veamos los objetos que aparecen en más de un mosaico. Para cada uno de esos mosaicos, ¿cuántas exposiciones hubo?

En este ejemplo, utilizamos una subconsulta que en sí misma es una combinación de varias tablas.

```SQL
SELECCIONE t.nexp, f.tileid, q1.targetid, q1.n_assign DESDE desi_dr1.fiberassign COMO f
UNIRSE (SELECCIONAR ff.targetid, CONTAR(*) COMO n_assign DESDE desi_dr1.fiberassign COMO ff GRUPO POR ff.targetid) COMO q1 ON f.targetid = q1.targetid
ÚNETE a desi_dr1.tile AS t ON f.tileid = t.tileid LÍMITE 100;
```

In [ ]:
q = """SELECT t.nexp, f.tileid, q1.targetid, q1.n_assign
FROM desi_dr1.fiberassign AS f
JOIN (SELECT ff.targetid, COUNT(*) AS n_assign FROM desi_dr1.fiberassign AS ff GROUP BY ff.targetid) AS q1 ON f.targetid = q1.targetid
JOIN desi_dr1.tile AS t ON f.tileid = t.tileid LIMIT 100;

Ahora veamos cuál cree la tabla de corrimiento al rojo que es el número de exposiciones para estos objetos.

```SQL
SELECCIONE z.* DESDE desi_dr1.zpix COMO z DONDE z.targetid EN (6432023904256, 6448025174016[, ...]);
```

In [ ]:
q3 = """SELECT z.* FROM desi_dr1.zpix AS z
WHERE z.targetid IN ({0});

## ## Ejercicio

* ¿Cuál es la distribución del número de exposiciones?

## Emparejar tablas de desplazamiento al rojo con datos de orientación

Hacer coincidir los resultados del corrimiento al rojo con la orientación es una tarea importante para muchos proyectos científicos de DESI. A continuación describimos brevemente los fundamentos de este emparejamiento utilizando la base de datos.

## # Basado en mosaicos

La tabla "ztile" contiene una clave externa que apunta a la clave principal de la tabla "destino". Esto significa que para cualquier fila de `ztile`, hay una entrada previamente coincidente en la tabla `target` y se puede acceder a ella con una combinación simple, como por ejemplo:
```SQL
SELECCIONE z.targetid, z.spgrp, z.spgrpval, z.tileid, z.z, z.zwarn, z.spectype,
t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target,
p.ra, p.dec
DESDE desi_dr1.ztile COMO z
ÚNETE a desi_dr1.target AS t ON z.targetphotid = t.id
ÚNETE a desi_dr1.photometry COMO p ON z.targetid = p.targetid
LÍMITE 50;
```
Tenga en cuenta `z.targetphotid = t.id`. Esta consulta también demuestra una UNIÓN con la tabla "fotometría", donde "targetid" es único.

In [ ]:
q = """SELECT z.targetid, z.spgrp, z.spgrpval, z.tileid, z.z, z.zwarn, z.spectype,
    t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target,
    p.ra, p.dec
FROM desi_dr1.ztile AS z
JOIN desi_dr1.target AS t ON z.targetphotid = t.id
JOIN desi_dr1.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## # HEALBasado en Pixel

Los resultados del corrimiento al rojo de los coadds basados ​​en HEALPixel en la tabla `zpix` pueden estar compuestos por varios mosaicos, por lo que no es tan sencillo compararlos con los datos de orientación. En la mayoría de los casos, la información de orientación es la misma para un "targetid" en varios mosaicos, siempre y cuando los mosaicos formen parte de la misme survey (*p. ej., "sv1"). Sin embargo, no *siempre* es así, por lo que hemos recopilado los mejores valores de la información de orientación y los hemos colocado directamente en la tabla `zpix`. Aquí está la consulta análoga a la consulta anterior:
```SQL
SELECCIONE z.targetid, z.survey, z.program, z.healpix, z.z, z.zwarn, z.spectype,
z.sv1_desi_target, z.sv1_bgs_target, z.sv1_mws_target, z.sv1_scnd_target,
p.ra, p.dec
DESDE desi_dr1.zpix COMO z
ÚNETE a desi_dr1.photometry COMO p ON z.targetid = p.targetid
LÍMITE 50;
```

In [ ]:
q = """SELECT z.targetid, z.survey, z.program, z.healpix, z.z, z.zwarn, z.spectype,
    z.sv1_desi_target, z.sv1_bgs_target, z.sv1_mws_target, z.sv1_scnd_target,
    p.ra, p.dec
FROM desi_dr1.zpix AS z
JOIN desi_dr1.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## Progreso de el survey

Veamos qué noches tienen datos y contemos el número de exposiciones por noche.

```SQL
SELECCIONE e.night, CONTAR(e.expid) COMO n_exp FROM desi_dr1.exposure COMO e GRUPO POR e.night ORDEN POR e.night;
```

In [ ]:
q = "SELECT e.night, COUNT(e.expid) AS n_exp FROM desi_dr1.exposure AS e GROUP BY e.night ORDER BY e.night;"
night_exposures = qc.query(sql=q, fmt='pandas', timeout=600)
night_exposures

Marca de tiempo de observación para una noche determinada.  Observe que tenemos tanto MJD como un objeto `datetime.datetime` correspondiente en la base de datos.

```SQL
SELECCIONE e.expid, e.mjd, e.date_obs DESDE desi_dr1.exposure COMO e DONDE e.night = 20210428 ORDEN POR e.expid;
```

In [ ]:
q = "SELECT e.expid, e.mjd, e.date_obs FROM desi_dr1.exposure AS e WHERE e.night = 20210428 ORDER BY e.expid;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

Entonces, para un objetivo determinado en la tabla "objetivo", ¿cuándo se completó la observación?  En otras palabras, si un objetivo tiene múltiples observaciones, queremos la fecha de la *última* observación.  Primero, ¿cuántos objetivos hay?

```SQL
SELECCIONE CONTAR(*) COMO n_targets DESDE desi_dr1.target;
```

In [ ]:
q = "SELECT COUNT(*) AS n_targets FROM desi_dr1.target;"
N_targets = qc.query(sql=q, fmt='pandas', timeout=600)
N_targets

Ahora buscamos objetivos que tengan observaciones y encontramos el MJD de la observación.

```SQL
SELECCIONE f.targetid, e.expid, e.mjd DESDE desi_dr1.fiberassign COMO f
UNIRSE (SELECCIONAR tt.targetid DESDE desi_dr1.target AS tt UNIRSE a desi_dr1.fiberassign COMO ff ON tt.targetid = ff.targetid
ÚNETE a desi_dr1.exposure COMO ee ON ff.tileid = ee.tileid GRUPO POR tt.targetid) COMO q1 ON f.targetid = q1.targetid
ÚNETE a desi_dr1.exposure COMO e ON f.tileid = e.tileid ORDENAR POR q1.targetid, e.expid;
```

Tenga en cuenta que esto consta de una consulta interna, `(SELECT tt.targetid FROM desi_dr1.target AS tt ...) AS q1` y una consulta externa que trata `q1` como si fuera una tabla. En algunos casos, podemos ejecutar esta consulta exactamente como se escribió anteriormente, pero debido a limitaciones de tiempo, dividiremos esta consulta y la ejecutaremos de forma asincrónica, guardando los resultados intermedios en una tabla MyDB.

In [ ]:
#
# Limpia cualquier tabla intermedia existente.
#
overwrite_q1_table = False
q1_table = 'desi_dr1_survey_progress_q1'
mydb_tables = qc.mydb_list().split('\n')
if q1_table in mydb_tables and not overwrite_q1_table:
    print(f"Usando la tabla de progreso de el survey existente {q1_table}.")
else:
    qc.mydb_drop('desi_dr1_survey_progress_q1')
    #
    # Encuentre todos los objetivos que tengan observaciones.
    #
    q1 = """SELECT tt.targetid FROM desi_dr1.target AS tt JOIN desi_dr1.fiberassign AS ff ON tt.targetid = ff.targetid
        JOIN desi_dr1.exposure AS ee ON ff.tileid = ee.tileid GROUP BY tt.targetid;"""
    response = qc.query(sql=q1, out=f'mydb://{q1_table}', async_=True, wait=True, poll=60, verbose=True, timeout=6000)
    print(response)
#
# Encuentre las fechas de exposición (MJD) para los objetivos que se han observado
#
q2 = f"""SELECT f.targetid, e.expid, e.mjd FROM desi_dr1.fiberassign AS f
    JOIN mydb://{q1_table} AS q1 ON f.targetid = q1.targetid
    JOIN desi_dr1.exposure AS e ON f.tileid = e.tileid ORDER BY q1.targetid, e.expid;"""
response = qc.query(sql=q2, fmt='pandas', async_=True, wait=True, poll=60, verbose=True, timeout=6000)
targetid = response.targetid.values
expid = response.expid.values
mjd = response.mjd.values
#
# Utilice los recuentos para dar la *última* observación.
#
unique_targetid, i, j, c = np.unique(targetid, return_index=True, return_inverse=True, return_counts=True)
unique_expid = expid[i + (c-1)]
unique_mjd = mjd[i + (c-1)]

Ahora tenemos los objetivos y la fecha de la última observación.  Pero está ordenado por "targetid".

In [ ]:
ii = unique_expid.argsort()
unique_targetid, i3, j3, c3 = np.unique(unique_expid[ii], return_index=True, return_inverse=True, return_counts=True)
N_completed = np.cumsum(c3)

Ahora graficamos la fracción completada versus el tiempo.  Además, mostramos el número de exposiciones por noche.

In [ ]:
min_mjd = 10*(int(mjd.min())//10)
fig, axes = plt.subplots(1, 2, figsize=(16, 8), dpi=100)
p1 = axes[0].plot(unique_mjd[ii][i3] - min_mjd, N_completed/N_targets.n_targets.values[0], 'k-')
foo = axes[0].set_xlabel(f'MJD-{min_mjd:d}')
foo = axes[0].set_ylabel('Fracción completada')
foo = axes[0].grid(True)
foo = axes[1].bar(pd.to_datetime(night_exposures['night'],format="%Y%m%d"), night_exposures['n_exp'], color='black', width=1)
foo = axes[1].set_xlabel('Fecha')
foo = axes[1].set_ylabel('Número de exposiciones')

## # Ejercicio

* Desglose el progreso por clase de destino, bit de destino, etc.

## Usando q3c

[q3c](https://github.com/segasai/q3c) ([Koposov & Bartunov 2006](https://ui.adsabs.harvard.edu/abs/2006ASPC..351..735K/abstract)) es una biblioteca popular que proporciona indexación espacial y búsqueda en bases de datos astronómicas. Aquí demostraremos cómo acceder a esta funcionalidad. Esta es una búsqueda radial ("cono") en un punto arbitrario en la huella DESI:

```SQL
SELECCIONE p.*, z.*, q3c_dist(p.ra, p.dec, 180.0, 0.0) AS distancia_radial
DESDE desi_dr1.photometry AS p ÚNETE a desi_dr1.zpix AS z ON p.targetid = z.targetid
DONDE q3c_radial_query(p.ra, p.dec, 180.0, 0.0, 1.0/60.0); -- 1 minuto de arco
```

In [ ]:
q = """SELECT p.*, z.*, q3c_dist(p.ra, p.dec, 180.0, 0.0) AS radial_distance
    FROM desi_dr1.photometry AS p JOIN desi_dr1.zpix AS z ON p.targetid = z.targetid
    WHERE q3c_radial_query(p.ra, p.dec, 180.0, 0.0, 1.0/60.0);"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## # Ejercicio

* ¿Qué espectros hay cerca de tu objeto favorito?

## Recursos y referencias

* [Notebook de base de datos DESI](https://github.com/desihub/tutorials/blob/main/database/spectroscopic-production-database.ipynb) (*nota*: requiere acceso a [NERSC](https://www.nersc.gov)).
* [Documentación de la base de datos DESI](https://data.desi.lbl.gov/doc/access/database/).
* [Documento de publicación temprana de datos de DESI](https://arxiv.org/abs/2306.06308).